In [ ]:
import pandas as pd 
import numpy as np
import torch
import torch.nn as nn
import torch.nn.functional as F
from sklearn.metrics import accuracy_score
print(torch.__version__)

1.6.0+cu101


In [ ]:
np.random.seed(123)
torch.manual_seed(123)

In [ ]:
class clf_torch(nn.Module):
  def __init__(self):
       super().__init__()
       self.dense0 = nn.Linear(30,8)
       torch.nn.init.normal_(self.dense0.weight, mean = 0.0, std=0.05)
       self.dense1 = nn.Linear(8,8)
       torch.nn.init.normal_(self.dense1.weight, mean = 0.0, std=0.05)
       self.dense2 = nn.Linear(8,1 )
       self.activation = nn.ReLU()
       self.dropout = nn.Dropout(0.2)
       self.output = nn.Sigmoid()

  def forward(self, X):
       X = self.dense0(X)
       X = self.activation(X)
       X = self.dropout(X)
       X = self.dense1(X)
       X = self.activation(X)
       X = self.dropout(X)
       X = self.dense2(X)
       X = self.output(X)
       return X 

In [ ]:
clf = clf_torch()

In [ ]:
state_dict = torch.load("/content/checkpoint.pth")

In [ ]:
state_dict

OrderedDict([('dense0.weight',
              tensor([[-6.2938e-01, -6.3880e-02, -3.8812e-01, -1.0131e-01,  5.3613e-02,
                       -5.2737e-02,  5.9786e-02, -6.1688e-02,  2.9235e-02,  8.3439e-02,
                        5.2361e-04,  1.0154e-03,  6.2317e-04, -1.3866e-03,  1.9200e-02,
                        1.1764e-01, -3.8250e-01, -2.6600e-01, -6.0280e-03,  4.2274e-01,
                       -4.7407e-01, -3.1142e-02, -3.2840e-01,  1.8073e-01,  4.6571e-02,
                        4.2041e-02,  2.2236e-02,  4.5930e-02,  3.1673e-02,  6.3382e-01],
                      [ 8.2932e-02,  1.6685e-02,  5.3594e-02, -5.2358e-02,  8.3542e-02,
                       -2.1864e-04,  4.8437e-02, -1.6590e-03, -7.1115e-02, -2.8024e-02,
                        1.3979e-02, -1.1546e-02, -4.8893e-02,  1.0772e-01, -1.0195e-14,
                        1.0536e-04,  1.7714e-02, -3.6042e-13, -3.3771e-03, -4.1130e-14,
                       -5.3232e-02,  4.5065e-02, -4.9866e-02, -6.6553e-02, -8.3494e-02,


In [ ]:
clf.load_state_dict(state_dict)

<All keys matched successfully>

In [ ]:
novo = torch.tensor([[15.80, 8.34, 231, 900, 0.10, 0.26, 0.08, 0.134, 0.178,
                  0.1, 0.05, 1098, 0.87, 6789, 165.2, 0.005, 0.04, 0.05, 0.015,
                  0.08, 0.007, 23.15, 16.64, 178.5, 2018, 0.14, 0.185,
                  0.84, 158, 0.363]], dtype = torch.float)

In [ ]:
clf.eval()

clf_torch(
  (dense0): Linear(in_features=30, out_features=8, bias=True)
  (dense1): Linear(in_features=8, out_features=8, bias=True)
  (dense2): Linear(in_features=8, out_features=1, bias=True)
  (activation): ReLU()
  (dropout): Dropout(p=0.2, inplace=False)
  (output): Sigmoid()
)

In [ ]:
previsao = clf(novo)
previsao = (previsao.detach().numpy() > 0.5)
print(previsao)

[[ True]]


In [ ]:
previsores = pd.read_csv("entradas_breast.csv")
classe = pd.read_csv("saidas_breast.csv")


In [ ]:
previsores = torch.tensor(np.array(previsores), dtype=float)
classe = torch.tensor(np.array(classe), dtype=float)

In [ ]:
predict = clf.forward(previsores.float()).detach()

In [ ]:
predict

tensor([[3.3624e-09],
        [3.0390e-06],
        [1.5145e-04],
        [3.5194e-06],
        [2.2234e-03],
        [4.7528e-01],
        [1.4094e-06],
        [1.3330e-01],
        [2.5540e-01],
        [1.8320e-03],
        [4.2397e-02],
        [8.5288e-04],
        [9.9827e-02],
        [8.8744e-01],
        [8.8744e-01],
        [3.5808e-01],
        [5.0096e-03],
        [3.0901e-04],
        [4.2124e-11],
        [8.9949e-01],
        [8.8744e-01],
        [1.0000e+00],
        [3.0697e-01],
        [1.4131e-11],
        [1.2919e-11],
        [6.7497e-05],
        [3.5849e-01],
        [5.9923e-03],
        [1.7780e-04],
        [1.0012e-01],
        [1.0846e-04],
        [1.5637e-02],
        [2.1437e-03],
        [1.6319e-06],
        [5.0770e-04],
        [3.2573e-02],
        [8.3584e-01],
        [9.9843e-01],
        [7.8532e-01],
        [5.6025e-01],
        [6.6390e-01],
        [8.8744e-01],
        [4.0303e-06],
        [7.4666e-02],
        [7.9646e-01],
        [1

In [ ]:
F.binary_cross_entropy(predict, classe.float()).numpy()

array(0.1673266, dtype=float32)

In [ ]:
def My_accuracy_score(y_true, y_pred):
    y_pred = np.concatenate(tuple(y_pred))
    y_true = np.concatenate(tuple([[t for t in y] for y in y_true])).reshape(y_pred.shape)
    return (y_true == y_pred).sum() / float(len(y_true))

In [ ]:
My_accuracy_score(classe.numpy(),predict.numpy())

0.10193321616871705

In [ ]:
accuracy_score(classe.numpy(),predict.numpy())

0.10193321616871705